In [2]:
import pandas as pd
import numpy as np
import requests
import textract
import json
from crossref.restful import Members
members = Members()
from crossref.restful import Works
works = Works() #could you just alias instead? 
from sklearn.utils import shuffle

API key for Crossref - 1fd3ca24-3bcec314-2e981223-6589eaa9

Ok, first, build a tool that pops through crossref and get metadata (including DOIs) for a whole set of search terms and for a specific publisher name. 

Need to add a '&mailto=jononorato@gmail.com' as an end to the URL request to get us into the "polite" pool.

In [ ]:
new_meta_search(['Science', 'Chemistry'])

Before the url request
After the URL request
Percent complete: 0.11080786691531952
Successful request from the server!
Percent complete: 0.22161573383063904
Successful request from the server!
Percent complete: 0.33242360074595856
Successful request from the server!
Percent complete: 0.4432314676612781
Successful request from the server!
Percent complete: 0.5540393345765976
Successful request from the server!
Percent complete: 0.6648472014919171
Successful request from the server!
Percent complete: 0.7756550684072366
Successful request from the server!
Percent complete: 0.8864629353225562
Successful request from the server!
Percent complete: 0.9972708022378757
Successful request from the server!
Percent complete: 1.1080786691531952
Successful request from the server!
Percent complete: 1.2188865360685148
Successful request from the server!
Percent complete: 1.3296944029838342
Successful request from the server!
Percent complete: 1.4405022698991539
Successful request from the server!
Per

In [12]:
def new_meta_search(search_terms, publisher_name = 'Wiley'):
    """"""
    chem_subset = works.query(search_terms)
    pub = next(iter(members.query(publisher_name)))
    pub_id = pub['id']
    chem_subset = chem_subset.filter(member = pub_id)
    doc_number = chem_subset.count()
    print("Before the url request")
    url= chem_subset.url + "&select=DOI,link&rows=1000&mailto=jononorato@gmail.com&cursor="
    #This is our starting cursor value. It will be updated in the loop with each request we make.
    #Standard request will look like:
    #response = requests.get(url+cursor).json()
    cursor = '*'
    url_list = []
    doi_list = []
    saved_docs = 0
    #Build a dataframe to save everything in. 
    full_publist = pd.DataFrame()

    
    #We'll need to make the first request before we get into the while loop for the first time.
    response = requests.get(url+cursor).json()
    print("After the URL request")
    #len(quick_response['message']['items']) - That's how we check if the response was empty. 
    
    #We need to check to see if the response is empty (ie, has no links or DOIs) as that means we pulled all responses.
    #Needs nested loops. Outer loop is while len(response[message or wahtever]) > 0, 
    #Then inner loop is for loop with for entries in response['message']
    
    while len(response['message']['items']) > 0:
        #Add the total number of papers from the response to the saved docs list. 
        saved_docs += len(response['message']['items'])
        print("Percent complete: " + str(100*saved_docs/doc_number))
        
        #Ok, for every entry in the response, loop through each entry. 
        for entries in response['message']['items']:
            #Check to see if the response item has a link in it
            keycheck = True
            try:
                entries['link']
            except KeyError:
                keycheck = False
            if keycheck:
                #If the link exists, then append stuff
                URL = entries['link'][0]['URL']
                DOI = entries['DOI']
                #Update all http to https
                if URL[:5] != "https" and URL[:4] == 'http':
                    URL = 'https' + URL[4:]
                #Check to see if the URL format is correct. If it is, add it, otherwise you don't add it to the list.
                if URL[8] == 'a':
                    url_list.append(URL)
                    doi_list.append(DOI)
        
        #Build a dataframe of URLs and DOIs from our requests    
        full_publist = pd.DataFrame()
        full_publist['URL'] = url_list
        full_publist['DOI'] = doi_list
        full_publist.to_json(r'/Users/Jonathan/Desktop/master_wiley_list.json')
        
        #Ok, no we need to update the cursor, and make a new request for a new response. 
        cursor = response['message']['next-cursor']
        response_nojson = requests.get(url+cursor)
        #NEED TO CHECK THE STATUS CODE FROM THIS RESPONSE.
        #IF THE REQUEST FAILS, I SHOULD MAKE A SAVE BEFORE QUITTING.
        if response_nojson.status_code == 200:
            print("Successful request from the server!")        
        else:
            print("Status code is bad! The response code is: " + str(response_nojson.status_code))
            break
                
    #Build a dataframe of URLs and DOIs from our requests    
    full_publist['URL'] = url_list
    full_publist['DOI'] = doi_list
    
    #Save the full list somewhere on computer, so we have a master copy somewhere.
    full_publist.to_json(r'/Users/Jonathan/Desktop/master_wiley_list.json')

    
    return full_publist

In [38]:
def article_meta_search(search_terms, publisher_name = 'Wiley', restart = False, current_data = None):
    """A method to collect the metadata from crossref based on specific search terms passed in. The results
    are filtered down to a specific publisher. It then saves all the URLs as a single list."""
    #First, we query crossref, and pull the metadata for specific search terms, and filter by publishers.
    chem_subset = works.query(search_terms)
    pub = next(iter(members.query(publisher_name)))
    pub_id = pub['id']
    chem_subset = chem_subset.filter(member = pub_id)  
    doc_length = chem_subset.count()
    
    #Next build a list of all the URLs, trying to fix all the URL formats if possible. 
    if restart:
        url_list = current_data['URL']
        DOI_list = current_data['DOI']
    else:
        url_list = []
        DOI_list = []
    
    saved_docs = 0
    for items in chem_subset:
        #We need some way to check if this thing has a key that says "link"
        #There are things published by Wiley that don't have links, like book chapters.
        keycheck = True
        saved_docs += 1
        if saved_docs % 1000 == 0:
            print("Current progress: " + str(100.0*saved_docs/doc_length))
            if saved_docs % 10000 == 0:
                full_pub_list = pd.DataFrame([url_list, DOI_list], columns = ["URL", "DOI"])
                with open('/Users/Jonathan/Desktop/temp_wiley_list.json','w') as file:
                    json.dump(full_pub_list, file)
        try:
            items['link']
        except KeyError:
            keycheck = False
        if keycheck:
            URL = items['link'][0]['URL']
            DOI = items['DOI']
            #Update all http to https
            if URL[:5] != "https" and URL[:4] == 'http':
                URL = 'https' + URL[4:]
            #Check to see if the URL format is correct. If it is, add it, otherwise you don't add it to the list.
            if URL[8] == 'a':
                url_list.append(URL)
                DOI_list.append(DOI)
    #Make the two lists into a dataframe
    full_pub_list = pd.DataFrame([url_list, DOI_list], columns = ["URL", "DOI"])
    #Save the full list somewhere on computer, so we have a master copy somewhere.
    with open('/Users/Jonathan/Desktop/master_wiley_list.json','w') as file:
        json.dump(full_pub_list, file)
    
    return full_pub_list

Ok, once we have a list of Wiley URLs, we need to extract the fulltext URLs. We should plan to save current list and pass a current list. 

In [ ]:
def get_fulltext(input_filepath, save_path):
    """"""
    #You should update this token with your own API key from Wiley
    headers = {'CR-Clickthrough-Client-Token': "1fd3ca24-3bcec314-2e981223-6589eaa9", 
               "User-Agent": "Jon Onorato, mailto:jononorato@gmail.com"}
    
    #Ok, now we need to shuffle everything so we know we don't have bias in our data
    full_df = pd.read_json(input_filepath)
    full_df = shuffle(full_df)
    url_list = full_df['URL']
    doi_list = full_df['DOI']
    
    #Ok, now start looping through the full list of URLs
    while len(url_list) > 0:
        #Pull the first url and doi out of the list.
        url = url_list.pop()
        doi = doi_list.pop()
        #Query the Wiley database. 
        file = requests.get(url = url, headers = headers, allow_redirects = True)
        #Check now to see if the request was successful.
        if file.status_code == 200:
            #Save the file directly to the desktop. We need to do this to open it elsewhere I think.
            open('/Users/Jonathan/Desktop/file.pdf', 'wb').write(file.content)
            #Now pull the text out of the pdf file. 
            textract_text = textract.process('/Users/Jonathan/Desktop/file.pdf')
            #Ok, now clean up the extracted text of any newline and return characters.
            string_textract = str(textract_text)
            no_n = string_textract.replace('\\n', '')
            no_r_or_n = no_n.replace('\\r', '')
            
            #And we save the text from the pdf file as a .txt. 
            with open(f'{save_path}/{doi}.txt', 'w') as file2:
                file2.write(no_r_or_n)
        
        #Should check to see the response code is not a 429, which is a too many requests error
        elif file.status_code == 429:
            print("Error! We've made too many Wiley requests!")
            #Add the url we were trying to search for back to the url_list
            url_list.append(url)
            doi_list.append(doi)
            #Re-save the full blown url_list to a file of some sort.
            current_wiley_list = pd.DataFrame()
            current_wiley_list['URL'] = url_list
            current_wiley_list['DOI'] = doi_list
            current_wiley_list.to_json(save_path+"current_wiley_status.json")
            print("The remaining URLs have been saved to the desktop. Try again later!")
            break
            
        else:
            print("There was a problem of some sort. Status code is: " + str(file.status_code))
            url_list.append(url)
            doi_list.append(doi)
            #Re-save the full blown url_list to a file of some sort.
            current_wiley_list = pd.DataFrame()
            current_wiley_list['URL'] = url_list
            current_wiley_list['DOI'] = doi_list
            current_wiley_list.to_json(save_path+"current_wiley_status.json")
            print("The remaining URLs have been saved to the desktop. Try again later!")            
            
    print("You've analyzed all of your URLs! Good job!")

Ok, now the next thing that I need to do is build a small module to update and pull the metadata from the front of each piece of full text, after it has been converted from a .pdf to a .txt form.

Also, maybe write a method that enables loading of data from the desktop.

In [ ]:
#This loads things from a json file on the desktop. Doppppppeeeeeeeeeeeee it workssssssss.
json_dataframe = pd.read_json("/Users/Jonathan/Desktop/master_wiley_list.json")
json_dataframe['URL'][2]

In [ ]:
#Ok, lets figure out how to deal with the crap that's present in the pdfs. 
#There's a ton of /r and /n from newline and return commands. Maybe I can get rid of those?
